In [37]:
from sklearn.neighbors import NearestNeighbors
from  sklearn.feature_extraction.text import CountVectorizer
import DataFrameModel as dfm
import pandas as pd

In [13]:
data =  dfm.GetFrame()

Connexion réussie à la base de données
Connection fermer


In [97]:
#Traiter collone runtimes
dataModif = data
print(f"Total : {len(dataModif)}")
print(f"runtime Null : {len(dataModif[dataModif["runTimemins"]=="\\N"])}")
print(f"Restant : {len(dataModif[dataModif["runTimemins"]!="\\N"])}")

dataModif = dataModif[dataModif["runTimemins"]!="\\N"]
dataModif["runTimemins"] = dataModif["runTimemins"].apply(int)

Total : 187746
runtime Null : 18171
Restant : 169575


C:\Users\Candidat\AppData\Local\Temp\ipykernel_20120\1259155171.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataModif["runTimemins"] = dataModif["runTimemins"].apply(int)


In [98]:
#Launch year en int pas de valeur null due a la selection SQL
dataModif["launchYear"] = dataModif["launchYear"].apply(int)

C:\Users\Candidat\AppData\Local\Temp\ipykernel_20120\1118141606.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataModif["launchYear"] = dataModif["launchYear"].apply(int)


In [99]:
dataModif["numVotes"] = dataModif["numVotes"].apply(int)

C:\Users\Candidat\AppData\Local\Temp\ipykernel_20120\3593254790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataModif["numVotes"] = dataModif["numVotes"].apply(int)


In [58]:
# Liste de tous les genres pour avoir une idée du nombre de colonnes dans un Vectorizer
dataModif["genre_1"].value_counts()

genre_1
Drama          43865
Documentary    34579
Comedy         32647
Action         15801
Horror          9414
Biography       6793
Crime           6530
Adventure       5524
Thriller        3421
Animation       2083
\N              1860
Romance         1733
Family          1110
Fantasy          981
Mystery          899
Sci-Fi           778
Music            490
Musical          322
History          268
Sport            170
Western          155
War              118
Reality-TV        22
News               6
Talk-Show          6
Name: count, dtype: int64

In [20]:
dataModif["genre_2"].value_counts()

genre_2
               85553
Drama          22831
Romance         8245
Thriller        6670
Documentary     6096
Horror          5402
Comedy          4773
Crime           4424
Mystery         3799
Family          3430
Adventure       3074
History         2855
Music           2556
Fantasy         2455
Sci-Fi          1869
Animation       1464
Sport           1444
War              832
Musical          740
Biography        562
News             262
Western          215
Reality-TV        14
Talk-Show          9
Game-Show          1
Name: count, dtype: int64

In [100]:
# Drop /N dans genre_1 car c'est une valeur null
dataModif = dataModif[dataModif["genre_1"]!="\\N"]

In [101]:
# Fusion des 3 colonnes genre en une genres en seprant par un espace pour preparer le countvectorizer
dataModif["genres"] = dataModif[["genre_1", "genre_2","genre_3"]].agg(" ".join,axis=1)

In [102]:
# Suppression des colonnes inutiles
dataModif = dataModif.drop(["genre_1", "genre_2","genre_3"], axis=1)

In [103]:
# creation d'un objet vectorizer
vectorizerGenres = CountVectorizer()
# definition de la variable : les donnees a vectorizer
toVectGenres = dataModif["genres"]
# Prepare le vectorizer, les colonnes necessaires a la transformation
vectorizerGenres.fit(toVectGenres)
# compte les valeurs pour chaque ligne et chaque colonne cree par le fit
vectedGenres = vectorizerGenres.transform(toVectGenres)

In [104]:
# Converti le sparsematrice en dataframe pour la visualisation
dfvectedGenres = pd.DataFrame(vectedGenres.toarray(), columns=vectorizerGenres.get_feature_names_out())

In [105]:
# Fusion du dataframe d'origine avec les resultats du countvectorizer
dataModif = pd.concat([dataModif, dfvectedGenres], axis=1)

In [106]:
#On drop tout les NA . Question : pourquoi il y a 10% de NA dans toutes les collonnes creer avec concat
dataModif.dropna(inplace=True)


In [91]:
# creation d'un objet vectorizer
vectorizerTitres = CountVectorizer()
# definition de la variable : les donnees a vectorizer
toVectTitres = dataModif["movieName"]
# Prepare le vectorizer, les colonnes necessaires a la transformation
vectorizerTitres.fit(toVectTitres)
# compte les valeurs pour chaque ligne et chaque colonne cree par le fit
vectedTitres = vectorizerTitres.transform(toVectTitres)

In [79]:
# Converti le sparsematrice en dataframe pour la visualisation
dfvectedTitres = pd.DataFrame(vectedTitres.toarray(), columns=vectorizerTitres.get_feature_names_out())

MemoryError: Unable to allocate 86.4 GiB for an array with shape (149913, 77347) and data type int64

In [107]:
dataModif = dataModif.reset_index()

In [113]:
dataTrain = dataModif.drop(columns=["movieName","index","genres"])

In [111]:
model = NearestNeighbors(n_neighbors=5,metric="cosine")


In [ ]:
dataModif

In [114]:
model.fit(dataTrain)

NearestNeighbors(metric='cosine')

In [ ]:
film_id = 18
distance, indices = model.kneighbors([dataModif.iloc[film_id]], n_neighbors=6)